# Title: WES_237A_lab5_report
# PID: A69028485
# Name: William Wei
# GitHub username: William Lien Chin Wei
# Email (registered with GitHub): williams19834812@gmail.com

## Connect the PMOD_AD2 peripheral to PMODA.

# Using PYNQ library for PMOD_ADC

This just uses the built in Pmod_ADC library to read the value on the PMOD_AD2 peripheral.

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import Pmod_ADC    # Pmod_ADC library
base = BaseOverlay("base.bit")

In [2]:
adc = Pmod_ADC(base.PMODA)

Read the raw value and the 12 bit values from channel 1.

Refer to docs: https://pynq.readthedocs.io/en/v2.1/pynq_package/pynq.lib/pynq.lib.pmod.html#pynq-lib-pmod

In [3]:
adc.read_raw(ch1=1, ch2=1, ch3=0) # Get the raw value from the Pmod ADC.

[3665, 4095]

In [4]:
adc.read(ch1=1, ch2=1, ch3=0)     # Get the voltage from the Pmod ADC.

[1.7905, 1.9995]

# Using MicroblazeLibrary

Here we're going down a level and using the microblaze library to write $\textbf{I2C commands}$ directly to the $\textbf{PMOD_AD2 peripheral}$

Use the documentation on the PMOD_AD2 to answer lab questions

In [5]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
base = BaseOverlay("base.bit")

In [6]:
liba = MicroblazeLibrary(base.PMODA, ['i2c'])

In [7]:
dir(liba) # list the available commands for the liba object

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_constants',
 '_build_functions',
 '_mb',
 '_populate_typedefs',
 '_rpc_stream',
 'active_functions',
 'i2c_close',
 'i2c_get_num_devices',
 'i2c_open',
 'i2c_open_device',
 'i2c_read',
 'i2c_write',
 'release',
 'reset',
 'visitor']

In the cell below, open a new i2c device. Check the resources for the i2c_open parameters

# What command opens a new i2c device in the MicroblazeLibrary? What are the two parameters to this command?

<font color='blue'> Command i2c_open(sda, scl) opens an I2C device attached to an I/O switch configured to use the specified pins. Calling this function will disconnect any previously assigned pins and return them to a high-impedance state. </font>

In [8]:
sda, scl = 3, 2  # Clock line (SCL) and Data line (SDA)
device = liba.i2c_open(sda, scl)# TODO open a device

In [9]:
dir(device) # list the commands for the device class

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_func',
 'close',
 'get_num_devices',
 'open',
 'open_device',
 'read',
 'val',
 'write']

Below we write a command to the I2C channel and then read from the I2C channel. Change the buf[0] value to select different channels. See the AD spec sheet Configuration Register. https://www.analog.com/media/en/technical-documentation/data-sheets/AD7991_7995_7999.pdf

Changing the number of channels to read from will require a 2 byte read for each channel!

# What does 0x28 refer to in the following line, device.write(0x28, buf, 1)?

<font color='blue'>  It refers to $\textbf{slave_address}$. <br>
        
i2c_write(i2c dev_id, unsigned int slave_address, unsigned char* buffer, unsigned int length)
Issue a write command to the specified slave. <br>

i2c_read(i2c dev_id, unsigned int slave_address, unsigned char* buffer, unsigned int length)
Issue a read command to the specified slave. buffer is an array allocated by the caller of at least length length. <br>
</font>

# Why do we write and then read when using the Microblaze Library compared to just reading in the PMOD Library?
<font color='blue'>  
We write  i2c_write(i2c dev_id, unsigned int slave_address, unsigned char* buffer, unsigned int length)
Issue a write command to the specified slave. <br>

i2c_read(i2c dev_id, unsigned int slave_address, unsigned char* buffer, unsigned int length)
Issue a read command to the specified slave. buffer is an array allocated by the caller of at least length length. <br>
</font>

In [34]:
buf = bytearray(2)            # return b'\x00\x00', returns a bytearray object
buf[0] = int('00000000', 2)
device.write(0x28, buf, 1)
device.read(0x28, buf, 2)
#print(format(int(((buf[0] << 8) | buf[1]))))
print(format(int(((buf[0] << 8) | buf[1])), '#018b'))
#print(((buf[0] << 8) | buf[1]))

0b0000000100100101


Compare the binary output given by ((buf[0]<<8) | buf[1]) to the AD7991 spec sheet. You can select the data only using the following command

In [33]:
result_12bit = (((buf[0] & 0x0F) << 8) | buf[1])
print(result_12bit)

292


# Using MicroBlaze

In [29]:
base = BaseOverlay("base.bit")

In [30]:
%%microblaze base.PMODA

#include "i2c.h"

int read_adc(){
    i2c i2c_device = i2c_open(3, 2);
    unsigned char buf[2];
    buf[0] = 0;
    i2c_write(i2c_device, 0x28, buf, 1);
    i2c_read(i2c_device, 0x28, buf, 2);
    return ((buf[0] & 0x0F) << 8) | buf[1];
}

In [31]:
read_adc()

293

# What is the difference between writing to the device when using the Microblaze Library and directly on the Microblaze?

<font color='blue'> It is the same writing to the device when using the Microblaze Library and directly on the Microblaze.</font>